In [12]:
# Install Gradio (for building the web interface)
!pip install gradio


In [13]:
# In-memory store for logs and agent interactions
import time

class SharedMemory:
    def __init__(self):
        self.records = []

    def store_record(self, record):
        record['timestamp'] = time.strftime("%Y-%m-%d %H:%M:%S")
        self.records.append(record)

    def get_all_records(self):
        return self.records

memory = SharedMemory()


In [14]:
# Classifier to detect format and intent
import json
import re

def detect_format(data):
    if isinstance(data, dict):
        return "JSON"
    elif isinstance(data, str):
        return "Email"
    else:
        return "Unknown"

def detect_intent(text_or_json):
    if isinstance(text_or_json, dict):
        text = json.dumps(text_or_json).lower()
    else:
        text = text_or_json.lower()

    if "invoice" in text:
        return "Invoice"
    elif "quote" in text or "rfq" in text:
        return "RFQ"
    elif "complaint" in text:
        return "Complaint"
    elif "regulation" in text:
        return "Regulation"
    else:
        return "General"


In [15]:
# Agent logic for handling JSON and Email formats
def process_json_agent(data):
    expected_keys = ["id", "date", "amount", "client"]
    missing = [k for k in expected_keys if k not in data]
    anomalies = []

    if "amount" in data and not isinstance(data["amount"], (int, float)):
        anomalies.append("amount should be a number")

    memory.store_record({"agent": "JSON Agent", "fields": data, "missing": missing})
    return {"agent": "JSON Agent", "normalized": data, "missing": missing, "anomalies": anomalies}

def process_email_agent(text):
    sender = re.search(r'From:\s*(.*)', text)
    subject = re.search(r'Subject:\s*(.*)', text)
    urgency = "High" if "urgent" in text.lower() or "asap" in text.lower() else "Normal"

    result = {
        "sender": sender.group(1) if sender else "Unknown",
        "subject": subject.group(1) if subject else "No Subject",
        "urgency": urgency,
        "intent": detect_intent(text)
    }

    memory.store_record({"agent": "Email Agent", "fields": result})
    return result


In [16]:
# This function handles both textbox and uploaded files
import os

chat_history = []

def chat_with_agents(user_input, uploaded_file):
    global chat_history

    content = None

    if uploaded_file:
        ext = os.path.splitext(uploaded_file.name)[1].lower()
        with open(uploaded_file.name, "r", encoding="utf-8") as f:
            file_content = f.read()
        if ext == ".json":
            try:
                content = json.loads(file_content)
            except:
                content = file_content
        else:
            content = file_content
    else:
        try:
            content = json.loads(user_input)
        except:
            content = user_input

    fmt = detect_format(content)
    intent = detect_intent(content)
    memory.store_record({"agent": "Classifier", "format": fmt, "intent": intent})

    response = f"**🧠 Classifier:** Format = `{fmt}`, Intent = `{intent}`"

    if fmt == "JSON":
        result = process_json_agent(content)
        response += "\n\n**📦 JSON Agent:**\n```json\n" + json.dumps(result, indent=2) + "\n```"
    elif fmt == "Email":
        result = process_email_agent(content)
        response += "\n\n**✉️ Email Agent:**\n```json\n" + json.dumps(result, indent=2) + "\n```"
    else:
        response += "\n\n❌ Unsupported format. Please enter valid JSON, email text, or upload a file."

    chat_history.append(("You", user_input if user_input else f"[Uploaded: {uploaded_file.name}]"))
    chat_history.append(("AI", response))

    return chat_history, "", None  # Clear textbox and file input


In [17]:
# Gradio interface with textbox + file upload + send button
import gradio as gr

with gr.Blocks(title="Multi-Agent AI Chat with File Upload") as demo:
    gr.Markdown("### 🤖 Multi-Agent AI Chat System with File Upload")

    chatbot = gr.Chatbot(height=400)
    with gr.Row():
        txt_input = gr.Textbox(placeholder="Paste JSON or email here...", lines=4, label="Text Input")
        file_input = gr.File(file_types=[".json", ".txt"], label="Upload File")
    send_btn = gr.Button("Send")

    send_btn.click(fn=chat_with_agents, inputs=[txt_input, file_input], outputs=[chatbot, txt_input, file_input])

demo.launch()


C:\Users\jhasn\AppData\Local\Temp\ipykernel_24744\1636207666.py:7: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=400)


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
